In [ ]:
from oncrit.utils.lanelet_operation import read_to_lanelet_map, show_lanelets, get_possible_paths, get_best_fitting_lanelet, get_centerline_of_lanelet
from oncrit.utils.visualization import draw_triangle

import matplotlib.pyplot as plt

## Loading the Map
Road information is given by the [Lanelet format](https://github.com/fzi-forschungszentrum-informatik/Lanelet2).
The helper function `show_lanelets()` draws the borderlines of a map onto a Matplotlib canvas, which is used for further visualization.

_After each visualisation (show/display call), the active figure is deleted. Accordingly, the road must be drawn again each time._

In [ ]:
osm_path = "../tests/data/k729_2022-03-16_fix.osm"
origin = (49.01160993928274,8.43856470258739)
fig = show_lanelets(osm_path=osm_path, origin=origin)

## Matching Traffic Participants to the Road
Traffic participants have to be assigned to a road element within the road network. This is the basis for generating future paths, which are derived from the road centrelines.
The matching/the alignment is done by the [Lanelet library](https://github.com/fzi-forschungszentrum-informatik/Lanelet2), which offers python bindings.

In the first step the lanelet map is loaded by `read_to_lanelet_map(osm_path, origin=origin)`
The `osm_path` is the same as used for the visualisation. This functions returns the lanelet map (`llet_map`) itself and a corresponding `routing_graph` which is later used for finding future paths.

In the following example 2 objects (`obj1, obj2`) are created. 
`lanelet2.matching.getDeterministicMatches`returns a list of potential lanelets (road segments) near to the position of the given objects. Note that it is possible that multiple fitting lanelets can be found. 

In [ ]:
import lanelet2
# load the lanelets as background
show_lanelets(osm_path=osm_path, origin=origin)

# loading the lanelet map
llet_map, routing_graph = read_to_lanelet_map(osm_path, origin=origin)

# obj1
x1, y1, psi_rad1 = 16.56, 10.71, 3.42
draw_triangle(x1, y1, psi_rad1, color="blue")
obj1 = lanelet2.matching.Object2d(1, lanelet2.matching.Pose2d(x1, y1, psi_rad1), [])
matches1 = lanelet2.matching.getDeterministicMatches(llet_map, obj1, 1.0)
print(matches1[0].lanelet.id, "Distance to obj1:", matches1[0].distance)

# obj2
x2, y2, psi_rad2 = 21.3,-20.0, 2.14
draw_triangle(x2, y2, psi_rad2, color="orange")
obj2 = lanelet2.matching.Object2d(2, lanelet2.matching.Pose2d(x2, y2, psi_rad2), [])
matches2 = lanelet2.matching.getDeterministicMatches(llet_map, obj2, 1.0)
print(matches2[0].lanelet.id, "Distance to obj2:", matches2[0].distance)


The function `get_best_fitting_lanelet` or `get_closest_lanelets_to_object` wraps the upper lanelet2 calls into a convenient macro.
The green line shows the centre lines of the two lanelets (road segments) on which obj1 and obj2 are located.
`get_centerline_of_lanelet` is a helper function to get the centerline.

In [ ]:
# load the lanelets as background
show_lanelets(osm_path=osm_path, origin=origin)

# Get closest lanelet to object
close_lanelet1 = get_best_fitting_lanelet(llet_map=llet_map, x=x1, y=y1, phi=psi_rad1)
close_lanelet2 = get_best_fitting_lanelet(llet_map=llet_map, x=x2, y=y2, phi=psi_rad2)
# Visualize Centerline
centerline_1 = get_centerline_of_lanelet(close_lanelet1)
centerline_2 = get_centerline_of_lanelet(close_lanelet2)
x_values_1, y_values_1 = zip(*centerline_1)
x_values_2, y_values_2 = zip(*centerline_2)
plt.plot(x_values_1, y_values_1, color="blue")
plt.plot(x_values_2, y_values_2, color="orange")

## Finding Possible Future Paths
The `get_possible_paths` function returns a list of possible paths along the centerline of the successive lanelets.
Each path is a tuple of x (`future_path[0]`) and y (`future_path[1]`) coordinates. Each path starts with the first point of the matched lanelet (`matches2[0].lanelet`).
The argument `max_routing_cost` defines the length when to stop searching, this has impact to the calculation speed.
If the endpoints of two trajectories are close (`< distance_threshold`), only one of them will be returned.
The argument `smooth_value` creates a moving window to smooth the path. `smooth_value=1` returns the centerline as it is.

In [ ]:
# load the lanelets as background
show_lanelets(osm_path=osm_path, origin=origin)

future_paths1 = get_possible_paths(
    matches1[0].lanelet,
    routing_graph,
    max_routing_cost=90,
    distance_threshold=5,
    smooth_value=3,
)

future_paths2 = get_possible_paths(
    matches2[0].lanelet,
    routing_graph,
    max_routing_cost=90,
    distance_threshold=5,
    smooth_value=3,
)

for future_path in future_paths1:
    plt.plot(future_path[0], future_path[1], color="blue")

for future_path in future_paths2:
    plt.plot(future_path[0], future_path[1], color="orange")

## Trimming the LineStrings

As can be seen in particular orange example, the center line of the lanelets begins well before the actual position of the traffic participant, relative to the course of the road.

In [ ]:
from oncrit.oncrit import Point, LineSegment, LineString, LineStringFan

_To increase the performance of the library, all operations are outsourced to rust from this step onwards._

The struct `LineStringFan` is provided for the collection of line strings.
The struct can be filled with the member function `from_vectors`.

In [ ]:
fan1 = LineStringFan.from_vectors(future_paths1)
fan2 = LineStringFan.from_vectors(future_paths2)

The `LineStringFans` can now be shortened using a `Point`.

For better performance, it is also recommended to optimise the `LineWtringFan` with the member function `optimize_linestrings()`. Here, redundant `LineSegment` are removed and only unique `Points` that do not overlap are retained in the `LinsstringFan`.

In [ ]:
# load the lanelets as background
show_lanelets(osm_path=osm_path, origin=origin)

p1 = Point(x1, y1)
p2 = Point(x2, y2)
fan1.project_and_cut(p1)
fan2.project_and_cut(p2)
fan1.optimize_linestrings()
fan2.optimize_linestrings()


list_of_linestrings1 = fan1.points_as_xy_vectors()
list_of_linestrings2 = fan2.points_as_xy_vectors()
for linestring in list_of_linestrings1:
    plt.plot(linestring[0], linestring[1], color="blue")
for linestring in list_of_linestrings2:
    plt.plot(linestring[0], linestring[1], color="orange")


draw_triangle(x1, y1, psi_rad1, color="blue")
draw_triangle(x2, y2, psi_rad2, color="orange")